# 1)**Introduction:** Why Product Simplification link here

In [2]:
from google.colab import files

# New data file upload. Do not execute this unless you want to change the  datafile.

#uploaded = files.upload() #Uncheck only if you want to upload a new file.

Saving mobileData.csv to mobileData.csv


# 2)	Data Gathering & Preparation: Explain key product data, handling multiple sources, establishing features.

In [20]:
import pandas as pd
import io
# creating data frame
df = pd.read_csv(io.BytesIO(uploaded['mobileData.csv']))
print(df.head())

  ProductID               ProductName  \
0     P0001      Mobile Postpaid Fish   
1     P0002      Mobile Prepaid Chair   
2     P0003  Mobile Postpaid Keyboard   
3     P0004        Mobile Prepaid Hat   
4     P0005     Mobile Postpaid Shoes   

                                  ProductDescription ProductType ServiceType  \
0  Mobile Postpaid plan for Residential customers...      Mobile    Postpaid   
1  Mobile Prepaid plan for Residential customers....      Mobile     Prepaid   
2  Mobile Postpaid plan for Business customers. 1...      Mobile    Postpaid   
3  Mobile Prepaid plan for Enterprise customers. ...      Mobile     Prepaid   
4  Mobile Postpaid plan for Enterprise customers....      Mobile    Postpaid   

  LifecycleStatus TargetCustomerSegment  Price Currency  BandwidthDataLimit  \
0        Obsolete           Residential  46.20      USD                  46   
1       In Design           Residential  54.71      USD                  15   
2          Active              Busi

# 3)	**Feature engineering:** Normalize feature to make it suitable for the AI training - Link here

In [21]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
def normalize_features(df):

    # Identify feature types : numerical, categorical, boolean and text features
    numerical_features = ['Price', 'BandwidthDataLimit', 'PS1001:DataLimit', 'PS1001:CallMinutes',
                        'PS2001:DownloadSpeed', 'PS2001:UploadSpeed', 'PS4001:StorageCapacity',
                        'PS4001:CameraResolution', 'PS2002:SLA']

    categorical_features = ['ServiceType', 'LifecycleStatus', 'TargetCustomerSegment',
                            'SalesChannel', 'GeographicAvailability', 'Unit',
                            'RS1001:ResourceType', 'SS2001:ServiceName', 'SR3001:ResourceStatus']

    boolean_features = ['PS1002:InternationalRoaming', 'PS3001:AdFree', 'PS2002:StaticIP']

    text_features = ['ProductDescription', 'TermsAndConditions', 'ProductFeatures', 'SR3001:ResourceID', 'SS2001:ServiceDescription']

    # Impute missing numerical values
    imputer_numerical = SimpleImputer(strategy='mean')
    df[numerical_features] = imputer_numerical.fit_transform(df[numerical_features])

    # Boolean to Numerical
    for feature in boolean_features:
        df[feature] = df[feature].fillna(False).astype(int)

    # Normalize numerical features
    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])

    # One-hot encode categorical features
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_categorical = encoder.fit_transform(df[categorical_features])
    encoded_columns = encoder.get_feature_names_out(categorical_features)
    encoded_df = pd.DataFrame(encoded_categorical, columns=encoded_columns)
    df = pd.concat([df, encoded_df], axis=1)

    # Text Feature Handling (TF-IDF)
    tfidf_vectorizer = TfidfVectorizer(max_features=100)
    for text_feature in text_features:
        tfidf_matrix = tfidf_vectorizer.fit_transform(df[text_feature].fillna(''))
        tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f'{text_feature}_tfidf_{i}' for i in range(tfidf_matrix.shape[1])])
        df = pd.concat([df, tfidf_df], axis=1)

    # Remove unnecessary/duplicate fields
    fields_to_remove = ['ProductName', 'Currency', 'RelatedProducts', 'ProductVersion',
                       'ProductType', 'IncludedServices','PO1001:OfferName','PO1001:OfferDescription','PO1001:MarketingCampaign'] + categorical_features + text_features

    df = df.drop(fields_to_remove, axis=1, errors='ignore')

    return df

df_normalized = normalize_features(df)
df_normalized.head(1)

,ProductID,Price,BandwidthDataLimit,PS1001:DataLimit,PS1001:CallMinutes,PS1002:InternationalRoaming,PS2001:DownloadSpeed,PS2001:UploadSpeed,PS3001:AdFree,PS4001:StorageCapacity,...,SR3001:ResourceID_tfidf_2,SR3001:ResourceID_tfidf_3,SR3001:ResourceID_tfidf_4,SR3001:ResourceID_tfidf_5,SR3001:ResourceID_tfidf_6,SR3001:ResourceID_tfidf_7,SS2001:ServiceDescription_tfidf_0,SS2001:ServiceDescription_tfidf_1,SS2001:ServiceDescription_tfidf_2,SS2001:ServiceDescription_tfidf_3
0,P0001,-0.624616,1.738799,1.738799,0.624033,1,-1.436556,-1.479591,0,-0.246321,...,0.0,0.0,0.0,0.0,0.0,0.0,0.377964,0.377964,0.377964,0.755929


Now let us makesure we have productID separately mapped and we create a dataframe for kmeans

In [22]:
product_ids = df["ProductID"]
kmeans_data = df.drop("ProductID",axis =1)
# Validate data
print(kmeans_data.isnull().sum()) #check for nulls
print(kmeans_data.describe()) #check for statistical anomolies.


ProductName                    0
ProductDescription             0
ProductType                    0
ServiceType                    0
LifecycleStatus                0
TargetCustomerSegment          0
Price                          0
Currency                       0
BandwidthDataLimit             0
Unit                           0
IncludedServices               0
SalesChannel                   0
GeographicAvailability         0
RelatedProducts                0
ProductVersion                 0
TermsAndConditions             0
ProductFeatures                0
PS1001:DataLimit               0
PS1001:CallMinutes             0
PS1002:InternationalRoaming    0
PS2001:DownloadSpeed           0
PS2001:UploadSpeed             0
PS3001:AdFree                  0
PS4001:StorageCapacity         0
PS4001:CameraResolution        0
PS2002:StaticIP                0
PS2002:SLA                     0
PO1001:OfferName               0
PO1001:OfferDescription        0
PO1001:MarketingCampaign       0
RS1001:Res

# 4)**Training**: Cluster products with similar characteristics, guidelines to evaluate the outcome

In [23]:

from sklearn.cluster import KMeans
# Set the number of clusters
k = 3

# Instantiate the KMeans model
kmeans = KMeans(n_clusters=k, random_state=42)  # Set random_state for reproducibility

# Train the model (fit the model to the data)
kmeans.fit(df_normalized.drop('ProductID', axis=1)) #remove product ID as it is not needed for training.

# Add the cluster labels to the DataFrame
df_normalized['cluster'] = kmeans.labels_

# Print the cluster assignments
print(df_normalized[['ProductID', 'cluster']])

# Optionally, print the cluster centers
# print("Cluster Centers:\n", kmeans.cluster_centers_)

# Map the clusters to the products.
df_original = pd.read_csv("mobileData.csv")
df_clustered = pd.merge(df_original, df_normalized[['ProductID', 'cluster']], on='ProductID')
df_clustered.to_csv("mobileData_clustered.csv")


  ProductID  cluster
0     P0001        1
1     P0002        2
2     P0003        0
3     P0004        2
4     P0005        0
5     P0006        1
6     P0007        2
7     P0008        0


Cluster validation:

In [24]:

from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity


# Cluster Size
print("Cluster Sizes:\n", df_clustered['cluster'].value_counts())

# Use the numerical_features list
numerical_features = ['Price', 'BandwidthDataLimit', 'PS1001:DataLimit', 'PS1001:CallMinutes',
                        'PS2001:DownloadSpeed', 'PS2001:UploadSpeed', 'PS4001:StorageCapacity',
                        'PS4001:CameraResolution', 'PS2002:SLA']

# Filter df_clustered to include only numerical and cluster columns
df_clustered_numeric = df_clustered[['cluster'] + numerical_features]

# Cluster Means
print("\nCluster Means:\n", df_clustered_numeric.groupby('cluster').mean())

# Silhouette Score
silhouette_avg = silhouette_score(df_normalized.drop(['ProductID', 'cluster'], axis=1), df_normalized['cluster'])
print("\nSilhouette Score:", silhouette_avg)

# WCSS
print("\nWCSS:", kmeans.inertia_)

# Example showing how to get the similarity matrix for one cluster.
cluster_number = 0
cluster_data = df_normalized[df_normalized['cluster'] == cluster_number].drop(['ProductID', 'cluster'], axis=1)
similarity_matrix = cosine_similarity(cluster_data)
print(similarity_matrix)

Cluster Sizes:
 cluster
2    3
0    3
1    2
Name: count, dtype: int64

Cluster Means:
          Price  BandwidthDataLimit  PS1001:DataLimit  PS1001:CallMinutes  \
cluster                                                                    
0        71.45           20.333333         20.333333           43.666667   
1        56.58           40.500000         40.500000           80.000000   
2        53.41           25.666667         25.666667           49.333333   

         PS2001:DownloadSpeed  PS2001:UploadSpeed  PS4001:StorageCapacity  \
cluster                                                                     
0                   24.000000           12.000000              149.333333   
1                   11.000000            5.500000               40.000000   
2                   19.333333            9.666667               42.666667   

         PS4001:CameraResolution  PS2002:SLA  
cluster                                       
0                      25.333333         0.0  
1   

Let's analyze the output:

**1. Cluster Sizes:**



1.   Cluster 0: 3 products
2.   Cluster 1: 2 products
3.   Cluster 2: 3 products

The clusters are relatively balanced.

**2. Cluster Interpretation:**

Cluster 0:
Highest average Price (71.45) and PS4001:StorageCapacity (149.33).
Highest PS2001:DownloadSpeed and PS4001:CameraResolution.
This cluster represents higher-end products with better performance and storage.

Cluster 1:
Lowest average Price (56.58), PS2001:DownloadSpeed, PS2001:UploadSpeed, and PS4001:StorageCapacity.
Highest PS1001:CallMinutes.
This cluster represents more basic products with a focus on call minutes.

Cluster 2:
Mid-range values for most features.
This cluster represents products with a balanced set of features.


**3. Silhouette Score:**

It indicates how well the clusters are separated and how well the similarity of the products within a cluster..
0.0983: Ideally we should be looking for higher Silhoette Score. We are getting a low score due to small samples in the demo file.
This is a very low silhouette score, indicating that the clusters are not well-separated.
Typically , this is a strong indicator that the K value should be changed, or that the data contains very little variance.

**4. WCSS (Within-Cluster Sum of Squares):**

65.86:
This value represents the sum of squared distances of samples to their closest cluster center. It's a useful metric for comparing different K-means models, but its absolute value is less informative without context.

**5. Similarity Matrix (Cluster 0):**

The similarity matrix shows the cosine similarity between the three products in cluster 0.
The first and third products have a relatively high similarity (0.61).
The other similarity scores are low.
This indicates that there is some similarity within cluster 0.


**6.   Similarity Analysis:**

The similarity matrix provides a more granular view of product
relationships within clusters.

Use this information to identify highly similar products for simplification.



# 5)	**Recommendations engine:** Show similar products to business while they are creating. So that they can take conscious decision.


As a next step we shall build a system which takes new products as input and give a list of similar products which are already present. This helps marketing team for decision making.

Select a file that contants new products in the same format as original file we used for training.

In [10]:
#new = files.upload() #uncheck only if you want to upload another file.

Saving NewProduct.csv to NewProduct.csv


In [25]:
import io
df_new = pd.read_csv(io.BytesIO(new['NewProduct.csv']))


In [26]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
def normalize_features(df):
    """Normalizes features for K-means clustering."""
    # ... (normalize_features function remains the same) ...
    numerical_features = ['Price', 'BandwidthDataLimit', 'PS1001:DataLimit', 'PS1001:CallMinutes',
                        'PS2001:DownloadSpeed', 'PS2001:UploadSpeed', 'PS4001:StorageCapacity',
                        'PS4001:CameraResolution', 'PS2002:SLA']

    categorical_features = ['ServiceType', 'LifecycleStatus', 'TargetCustomerSegment',
                            'SalesChannel', 'GeographicAvailability', 'Unit',
                            'RS1001:ResourceType', 'SS2001:ServiceName', 'SR3001:ResourceStatus']

    boolean_features = ['PS1002:InternationalRoaming', 'PS3001:AdFree', 'PS2002:StaticIP']

    text_features = ['ProductDescription', 'TermsAndConditions', 'ProductFeatures', 'SR3001:ResourceID', 'SS2001:ServiceDescription']

    imputer_numerical = SimpleImputer(strategy='mean')
    df[numerical_features] = imputer_numerical.fit_transform(df[numerical_features])

    for feature in boolean_features:
        df[feature] = df[feature].fillna(False).astype(int)

    scaler = StandardScaler()
    df[numerical_features] = scaler.fit_transform(df[numerical_features])

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    encoded_categorical = encoder.fit_transform(df[categorical_features])
    encoded_columns = encoder.get_feature_names_out(categorical_features)
    encoded_df = pd.DataFrame(encoded_categorical, columns=encoded_columns)
    df = pd.concat([df, encoded_df], axis=1)

    tfidf_vectorizer = TfidfVectorizer(max_features=100)
    for text_feature in text_features:
        tfidf_matrix = tfidf_vectorizer.fit_transform(df[text_feature].fillna(''))
        tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=[f'{text_feature}_tfidf_{i}' for i in range(tfidf_matrix.shape[1])])
        df = pd.concat([df, tfidf_df], axis=1)

    fields_to_remove = ['ProductName', 'Currency', 'RelatedProducts', 'ProductVersion',
                       'ProductType', 'IncludedServices','PO1001:OfferName','PO1001:OfferDescription','PO1001:MarketingCampaign'] + categorical_features + text_features

    df = df.drop(fields_to_remove, axis=1, errors='ignore')

    return df, scaler, numerical_features
def recommend_similar_products(new_product_features, df_normalized, kmeans, product_id):
    """Recommends similar products based on a new product's features."""

    new_product_cluster = kmeans.predict(new_product_features)[0]

    cluster_products = df_normalized[df_normalized['cluster'] == new_product_cluster].drop(['ProductID', 'cluster'], axis=1)

    similarity_scores = cosine_similarity(new_product_features, cluster_products)[0]

    product_ids = df_normalized.loc[df_normalized['cluster'] == new_product_cluster, 'ProductID'].tolist()
    similar_products = sorted(zip(product_ids, similarity_scores), key=lambda x: x[1], reverse=True)

    print(f"New product (ProductID: {product_id}) belongs to cluster {new_product_cluster}.")
    print("Similar products:")
    for product_id, similarity in similar_products:
        print(f"  ProductID: {product_id}, Similarity: {similarity:.4f}")

# Main execution
df = pd.read_csv("mobileData.csv")
df_normalized, scaler, numerical_features = normalize_features(df)

k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(df_normalized.drop('ProductID', axis=1))
df_normalized['cluster'] = kmeans.labels_


# Example creation of df_normalized_new
df_normalized_new = df_normalized.sample(1).drop(['cluster'], axis=1) #This is a sample creation.

# Iterate through the rows of df_normalized_new
for index, row in df_normalized_new.iterrows():
    product_id = df_normalized_new.loc[index, 'ProductID']
    new_product_features = row.drop('ProductID').to_frame().T #Create a dataframe from the row.
    recommend_similar_products(new_product_features, df_normalized, kmeans, product_id)
    print("-" * 20) #Seperate the results.

New product (ProductID: P0002) belongs to cluster 2.
Similar products:
  ProductID: P0002, Similarity: 1.0000
  ProductID: P0007, Similarity: 0.4769
  ProductID: P0004, Similarity: 0.3622
--------------------


Ends of Demonstration.